In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.corpora as corpora
import gensim
from datetime import datetime, timedelta
import time
import pickle
import pprint

In [2]:
# Import csv file of pre-processed, tokenized articles
df = pd.read_csv('250k_full_lda_185.csv')
#df['summary_words'] = [eval(item) for item in df['summary_words']]
df['article_words'] = [eval(item) for item in df['article_words']]

In [3]:
df.head(1)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic,other_topics
0,0,539903,539903,2017-06-01 00:00:00,2017,6.0,1,James Vincent,Misfit’s new tool lets you customize your fitn...,Misfit has launched a new tool that lets users...,https://www.theverge.com/circuitbreaker/2017/6...,Tech,The Verge,"[misfit, launch, new, tool, let, user, persona...",28,[128]


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df= pd.read_pickle('p25million.pkl')
df.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[repeats, aug, column, without, change, opinio..."
1,168893,168893,2016-04-08 15:50:02,2016,4.0,8,David Roberts,New music for you: Edna Vazquez plays her song...,"Once a month (or so), I'm bringing you an epis...",https://www.vox.com/culture/2016/4/8/11391828/...,NaN,Vox,"[month, bring, episode, pickathon, pumphouse, ..."
2,2001409,2001409,2016-09-09 00:35:26,2016,9.0,9,The Associated Press,One Wild Pitch Leads to Two Runs as Indians Be...,Sports Briefing The Indians scored two runs on...,http://www.nytimes.com/2016/09/09/sports/baseb...,sports,The New York Times,"[sports, briefing, indians, score, two, run, d..."
3,358141,358141,2018-03-01 09:09:08,2018,3.0,1,Ron Miller,Dropbox to add native G Suite integration in n...,It’s been an eventful week for Dropbox coming ...,https://techcrunch.com/2018/03/01/dropbox-to-a...,NaN,TechCrunch,"[eventful, week, dropbox, come, announcement, ..."
4,307943,307943,2017-12-14 00:00:00,2017,12.0,14,Christopher Matthews,Walmart will start giving salary advances to 1...,Walmart announced Wednesday that it will start...,https://www.axios.com/walm-1513267182-9615b668...,NaN,Axios,"[walmart, announce, wednesday, start, allow, w..."


In [12]:
def lda_compute(df,num_topics,passes=20):
    start = time.time()
    id2word = corpora.Dictionary(df['article_words'])
    
    articles = df['article_words']
    
    
    corpus = [id2word.doc2bow(article) for article in articles]
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics,
                                           passes=passes,random_state=4)
    
#    print(lda_model.print_topics())
    print('-' * 10)
    doc_lda = lda_model[corpus]
    
    lda_lists = [lda_model[article] for article in corpus]
    print(f'lda_lists[-1]: {lda_lists[-1]}')
    lda_coeffs = [[pair[1] for pair in article] for article in lda_lists]
    print(f'lda_coeffs[-1]: {lda_coeffs[-1]}')
    lda_inds = [[pair[0] for pair in article] for article in lda_lists]
    print(f'lda_inds[-1]: {lda_inds[-1]}')
    
    rogues = [i for i in range(len(lda_inds)) if len(lda_inds[i])==0]
    
    lda_lists = [lda_lists[i] for i in range(len(lda_lists)) if i not in rogues]
    lda_coeffs = [lda_coeffs[i] for i in range(len(lda_coeffs)) if i not in rogues]
    lda_inds = [lda_inds[i] for i in range(len(lda_inds)) if i not in rogues]
    
    df.drop(rogues,inplace=True,axis=0)
    
    
#    df_rogue = df.loc[rogues,:]
#    return df_rogue 
    
###    lda_argmax = [np.argmax(tup) if len(tup) < 0 else None for tup in lda_coeffs]
#    lda_args = [np.arsgort(tup)[::-1] for tup in lda_coeffs]
    
    lda_argmax = [np.argmax(tup) for tup in lda_coeffs]
    
    
    print(f'lda_argmax[-1]: {lda_argmax[-1]}')
#    lda_inds = [pair[0] for pair in lda_tuples]
#    print(lda_inds[-1])
#    print('-' * 10)
#    print(lda_argmax[-1])
#    print('-' * 10)
###    lda_topics = [lda_inds[i][lda_argmax[i]] if lda_argmax[i] else None for i in range(len(lda_lists))]
    lda_topics = [lda_inds[i][lda_argmax[i]] for i in range(len(lda_lists))]
    
    sec_threshold = 0.5
    
#    pdb.set_trace()
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
        
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
    
    
#    secondary topics = [[lda_inds[j] for j in range(len(lda_inds[i])) if lda_coeffs[j] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]      
    
    secondary_topics = [[pair[0] for pair in lda_lists[i] if 1 > pair[1] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]
    
    df['lda_topic'] = lda_topics
    
    df['other_topics'] = secondary_topics   
    
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df, lda_model, corpus#, doc_lda

In [9]:
#dft = df[:20000]
dft = df.copy()

In [10]:
len(dft)

300000

In [11]:
len(df)

300000

In [13]:
#16:30
dft, lda_model, corpus = lda_compute(dft,190)

Traceback (most recent call last):
  File "C:\Users\fergu\Anaconda3\lib\multiprocessing\queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "C:\Users\fergu\Anaconda3\lib\multiprocessing\reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
MemoryError


KeyboardInterrupt: 

In [37]:
# Save the model and df to desired filenames
dft.to_csv('test_df.csv')
lda_model.save('test_ldamodel')

## Word2vec Alternative to LDA

In [ ]:
def create_w2v(col1,col2,max_epochs,vec_size,window,min_count,resume=False):
    now  = datetime.now()
    start_time_str = now.strftime('%H:%M:%S')
    print('started at',start_time_str)
        
    start_time = time.time()
    word_list = [[words for words in line] for line in col1]
    unpacked_word_list = [word for line in word_list for word in line]
    
    filename = 'model.sav'
    
    if resume == False:
        model = Word2Vec(col1,vector_size=vec_size,window=window,min_count=min_count,workers=-1)
        model.train(col1,total_examples=len(col1),epochs=max_epochs)
        print('word2vec model has been trained.')
        pickle.dump(model,open(filename,'wb'))
    else:
        file = open(filename,'rb')
        model = pickle.load(file)
        
    averaged_vectors = [] if resume == False else pd.read_csv('intermediate_results.csv',index_col=0)
    
    if resume == True:
        averaged_vectors = [list(averaged_vectors.loc[val,:]) for val in averaged_vectors.index]
    
    j = start_index= len(averaged_vectors)
    if j != 0:
        print(f'{round(j/len(col1),3) * 100}% complete.')
    start_checkpoint_time = time.time()
    
    progress_indicators = []
    
    
    for row in range(j,len(col1)):
        list_of_words = col1[row]
        list_of_vectors = [model.wv[word] for word in list_of_words if unpacked_word_list.count(word) >= min_count]
        avg_vector = np.mean(list_of_vectors,axis=0) if list_of_vectors else np.zeros(vec_size)
        averaged_vectors.append(avg_vector)
#        print(avg_vector)
        
        j += 1
#        print(f'{j} done')
        if j % 1000 == 0:
            intermediate_results = pd.DataFrame(averaged_vectors)
            intermediate_results.to_csv('intermediate_results.csv')
            
        percent_complete = round(j/len(col1) * 100)
        if (percent_complete % 1 == 0) & (percent_complete not in progress_indicators):
            cum_time_raw = (time.time() - start_time)/60
            cum_time = round((time.time() - start_time)/60,1)
            print('{}% complete. Time elapsed: {} min'.format(percent_complete,cum_time))
            progress_indicators.append(percent_complete)
            
            remaining_time = cum_time_raw * (len(col1) - j) / (j - start_index)
            
            est_finish_time = datetime.now() + timedelta(minutes=remaining_time)
            est_finish_time = est_finish_time.strftime('%D:%H:%M:%S')
            print(f'Estimated finish time: {est_finish_time}')
            print('-' * 15)
            
    df_of_vectors = pd.DataFrame(averaged_vectors)
        
    intermediate_table = pd.concat([col2,df_of_vectors],axis=1)
        
    rows_to_remove = [ind for ind in intermediate_table.index if intermediate_table.iloc[ind,2] == 0]
        
    removed_comments = pd.DataFrame(col2[rows_to_remove])
    final_table = intermediate_table.drop(rows_to_remove,axis=0)
        
    interval = round((time.time() - start_time)/60,1)
    print('-' * 25)
    print(f'Finished. That took {interval} min.')
    now = datetime.now()
    end_time_str = now.strftime('%H:%M:%S')
    print(f'Actual finish time: {end_time_str}')
        
    return final_table, removed_comments

In [ ]:
final_table, removed_comments = create_w2v(tab['article_words'],tab['tweet'],5,100,5,5)